# WebSockets 

- We use WebSockets when we need real-time, two-way communication between a client (usually browser) and a server. Unlike traditional HTTP requests (which are one-way: client requests → server responds), WebSockets allow both the client and server to send messages to each other anytime, over a single, long-lived connection.

- Why WebSockets Are Used:

    - Real-Time Updates 
        - Ideal for applications like chat apps, live notifications, collaborative tools, stock tickers, multiplayer games. 
        - Server can push data to the client instantly (no need to poll).
    
    - Low Latency
        - No need to establish a new connection for every request (as in HTTP).
        - Reduces overhead and response times.
    
    - Two-Way Communication
        - Both client and server can send and receive data anytime.
        - Enables features like live typing indicators, real-time syncing, and feedback loops.
    
    - Efficient Data Exchange
        - Uses a lightweight protocol compared to HTTP.
        - Lower bandwidth usage since headers are smaller and persistent.
    
    - Persistent Connection
        - Once established, the connection stays open (until closed).
        - Avoids repeated handshakes of HTTP.

- When Not to Use WebSockets
    - For simple APIs or CRUD operations, use regular HTTP (REST or GraphQL).
    - For SEO-heavy content where search engine crawling is needed.
    - If the server doesn’t support persistent connections well.

- You are connecting to a WebSocket server (wss://ws.gomarket-cpp.goquant.io/ws/l2-orderbook/okx/BTC-USDT-SWAP).

- The connection is managed asynchronously using async with, ensuring the WebSocket connection opens and closes cleanly.

- You are entering an infinite loop (while True) that continuously receives messages (msg = await websocket.recv()) from the server, and the loop continues running indefinitely. (here we just doing for 5 sec)

# WebSocket Client for Orderbook Data

In [ ]:
import websockets
import asyncio
import json

In [24]:

orderbook_data = {'bids': [], 'asks': []}

async def fetch_orderbook_for_5s():
    url = "wss://ws.gomarket-cpp.goquant.io/ws/l2-orderbook/okx/BTC-USDT-SWAP" 
    try:
        async with websockets.connect(url) as websocket:
            start = asyncio.get_event_loop().time()
            while asyncio.get_event_loop().time() - start < 5:  # run for 5 seconds
                msg = await websocket.recv()
                data = json.loads(msg)
                if 'bids' in data and 'asks' in data:
                    orderbook_data['bids'] = data['bids']
                    orderbook_data['asks'] = data['asks']
                    top_bid = orderbook_data['bids'][0] if orderbook_data['bids'] else None
                    top_ask = orderbook_data['asks'][0] if orderbook_data['asks'] else None
                    # print(f"Top Bid: {top_bid} | Top Ask: {top_ask}")
                await asyncio.sleep(0.1)

    except Exception as e:
        print(f"Error: {e}")


# asyncio.run(fetch_orderbook_for_5s()) if script
# asyncio.run() tries to create a new event loop, but there’s already one running (e.g., Jupyter always has one).

await fetch_orderbook_for_5s()
orderbook_data

{'bids': [['103919.2', '756.21'],
  ['103919.1', '21.02'],
  ['103918.9', '0.13'],
  ['103918.4', '69.26'],
  ['103918', '4.59'],
  ['103917.1', '41.3'],
  ['103916.7', '41.4'],
  ['103916', '4.59'],
  ['103915', '0.01'],
  ['103914.7', '0.1'],
  ['103914.3', '0.62'],
  ['103914.1', '116'],
  ['103914', '4.59'],
  ['103913.1', '40.5'],
  ['103912.2', '52.19'],
  ['103912.1', '243.46'],
  ['103912', '123.2'],
  ['103911.7', '0.23'],
  ['103911.5', '0.02'],
  ['103911.4', '0.09'],
  ['103911.3', '33.75'],
  ['103910.9', '0.16'],
  ['103910.4', '0.49'],
  ['103910.3', '41.3'],
  ['103910.2', '0.09'],
  ['103910', '4.71'],
  ['103909.5', '0.01'],
  ['103909.4', '12'],
  ['103909.3', '137.28'],
  ['103909.2', '71.33'],
  ['103909.1', '21.9'],
  ['103908.9', '0.08'],
  ['103908.8', '0.03'],
  ['103908.5', '1.22'],
  ['103908.2', '0.01'],
  ['103908.1', '0.1'],
  ['103908', '4.82'],
  ['103907.9', '1.6'],
  ['103907.8', '39.05'],
  ['103907.1', '36.56'],
  ['103906.9', '1.09'],
  ['103906.8',

# Concepts 

- Volatility 
    - Volatility refers to the degree of variation in the price of an asset over time. The more volatile an asset is, the larger the price fluctuations will be within a short time period.

    - How Volatility Affects Slippage:
        - High volatility means the price is moving quickly and unpredictably. If you place an order in a volatile market, the price could change significantly before your order is executed.

        - Low volatility means the price is relatively stable and doesn't change as rapidly, which generally leads to lower slippage.
        
        - Example: Suppose you're trading Bitcoin (BTC). If the price of Bitcoin is moving rapidly between $30,000 and $31,000 in a short time, and you place a market order, your order might be filled at $31,200 due to the price movement during the execution process, causing slippage.
        On the other hand, in a low volatility market, the price might move only by $100 during the execution, and you might end up buying at $30,100 instead of $30,000, resulting in less slippage.

    - Volatility is often measured using metrics like Standard Deviation 

    - Reasons can be :
       
       - Breaking News : Sudden spikes/drops.

       - Low Liquidity : Price gaps
       
       - Large Orders : Price impact

    
- Liquidity 

    - Liquidity refers to how easily an asset can be bought or sold without causing a significant price change. It is determined by the depth of the order book (how many buy/sell orders are available) and the spread between the bid and ask prices.

    - How Liquidity Affects Slippage:
        
        - High liquidity means there are many orders at various price levels, which allows you to execute large trades without significantly affecting the price. In this case, slippage will be minimal.

        - Low liquidity means there are fewer orders available, and the price could shift drastically if a large trade is executed. This leads to higher slippage.

        - Example:  Suppose you're trading a highly liquid stock like Apple (AAPL). There are many orders at various price levels on both the buy and sell sides of the order book. If you place a market order to buy a large amount of stock, the price may only increase slightly, say by $0.05 due to the large number of available orders at various levels. Thus, the slippage is minimal.

        - However, if you're trading a low liquidity asset, such as a small-cap stock or a cryptocurrency with fewer market participants, a large order could push the price higher as there are not enough buy orders at the initial price. This would result in higher slippage. For example, a trade could cause the price to move $1.00 or more from the expected price.
    
    - Liquidity can be measured by:
        
        - Order book depth: The total volume of buy and sell orders available at various price levels.
        
        - Bid-ask spread: The difference between the highest bid (buy) and the lowest ask (sell) price. A larger spread indicates lower liquidity.

- Slippage 

    - Slippage is the difference between the expected price of a trade and the actual price at which the trade is executed.

    - Imagine you're at a market and see apples priced at ₹50/kg. You tell the shopkeeper, "I’ll take 5 kg," but in the time it takes him to start packing, the price changes to ₹52/kg. You end up paying more than expected — this is slippage.

    - You're trading a stock:
        
        - Expected Price: ₹100
        
        - Order Type: Market Order
        
        - Price When Order Hits Market: ₹102
        
        - Slippage = ₹102 - ₹100 = ₹2
    
    -  When is Slippage Most Likely?
       
       - Low liquidity : Not enough buyers/sellers at desired price.
       
       - High volatility	: Prices move rapidly due to news/events.
       
       - Large orders : Your trade can't be filled at one price and spills into higher/lower prices.
       
       - Market orders : Executed instantly, regardless of price changes.
    
    - Slippage = (Trade size in USD) * (Volatility Factor) * (1 + 1 / Liquidity)

# Model Implementation

Almgren-Chriss market impact model

- Purpose:
  - The Almgren-Chriss market impact model is designed to estimate the impact of a trade on the market price based on the trade's size. Larger trades tend to move the market more. This model helps quantify the market price impact based on trade size, volatility, and other parameters.

- If the trade size is 1000 USD, and the market impact formula calculates an impact of 0.5 USD, this means the price of the asset is expected to move by 0.5 USD due to the execution of the trade.
- If the market price was $50,000 for one Bitcoin, the new price after the trade could become $50,000.50.

In [25]:
def calculate_market_impact(trade_size_usd: float, alpha: float = 0.001, beta: float = 0.0001) -> float:  
    
    """
    Almgren-Chriss model to estimate market impact based on trade size.
    Parameters : 
         trade_size_usd: The trade size in USD
         alpha: Linear coefficient (default 0.001)
         beta: Quadratic coefficient (default 0.0001)
    Output : 
         The estimated market impact (how much price moves)
    """
         
    return alpha * trade_size_usd + beta * trade_size_usd**2

impact = calculate_market_impact(1000)
print(impact)



101.0


Regression models for slippage estimation

- Slippage refers to the difference between the expected price and the actual executed price of a trade.
- It occurs due to volatility or lack of liquidity.
-  We can estimate slippage using regression models based on features like trade size and market volatility.

In [3]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [12]:
class SlippageModel:

    def __init__(self):

        X = np.random.rand(1000,3) 
        trade_size = X[:,0]
        volatility = X[:,1]
        liquidity =  (1 - X[:, 2])  # lower liquidity -> higher slippage
        noise = np.random.rand(1000) # noise to avoid overfitting to a perfect pattern
        Y = trade_size * 0.01 + volatility * 0.02 +  liquidity*0.03 + noise*0.001

        # In real case : we will be training the model on real data.

        self.model = Sequential()
        self.model.add(Dense(64, input_dim=3, activation='relu')) 
        self.model.add(Dense(32, activation='relu')) 
        self.model.add(Dense(1, activation='linear')) 

        self.model.compile(optimizer=Adam(), loss='mean_squared_error')
        self.model.fit(X,Y, epochs=100, batch_size=10, verbose=0)
        
    def predict_slippage(self, trade_size_usd: float, volatility: float,liquidity : float) -> float:

        X_input = np.array([[trade_size_usd, volatility,liquidity]])  
        
        return self.model.predict(X_input)[0][0]  

In [13]:
slippage_model = SlippageModel()

predicted_slippage = slippage_model.predict_slippage(100, 0.5,0.2)
print(f"Predicted Slippage: {predicted_slippage}")


/Users/akashjain/Desktop/GoQuant_Simulator/GOquant/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted Slippage: 1.34266197681427


Maker/Taker proportion prediction

- In a trading market, makers add liquidity by placing limit orders, and takers take liquidity by executing market orders. 
- The maker/taker proportion can be used to predict how much liquidity will be added or taken based on trade conditions like trade size and volatility.


In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [16]:

class MakerTakerModel:

    def __init__(self):
        
        X = np.random.rand(1000,2)  # b/w 0 to 1
        trade_size = X[:,0]  
        volatility = X[:,1]
        noise = np.random.rand(1000) # noise to avoid overfitting to a perfect pattern
        Y = (1-trade_size) * 0.5 + (1 - volatility) * 0.3 + noise*0.1  # will be in range of 0 to 1
        # maker proportion roughly inversely related to trade size & volatility
        Y = np.clip(Y, 0, 1)  # ensure proportions are between 0 and 1

        self.model = Sequential()
        self.model.add(Dense(64, input_dim=2, activation='relu')) 
        self.model.add(Dense(32, activation='relu'))  
        self.model.add(Dense(1, activation='sigmoid'))  

        self.model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

        self.model.fit(X,Y, epochs=100, batch_size=10, verbose=0)

    def predict_maker_taker(self, trade_size_usd: float, volatility: float) -> float:

        X_input = np.array([[trade_size_usd, volatility]]) 
        return self.model.predict(X_input)[0][0] 

In [19]:

maker_taker_model = MakerTakerModel()

predicted_proportion = maker_taker_model.predict_maker_taker(1000, 0.5)
print(f"Predicted Maker/Taker Proportion: {predicted_proportion}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Maker/Taker Proportion: 0.0


Why neural networks ?

- Real-world financial data (like slippage or trading behavior) often has non-linear patterns.
- Example: A small trade may have low slippage, but a large one could have disproportionately high slippage.
- Neural networks are powerful at capturing such complex, non-linear mappings.
- With enough data, neural networks can generalize better than basic models.